In [1]:
# Mounting GCS to colab
# https://stackoverflow.com/questions/51715268/how-to-import-data-from-google-cloud-storage-to-google-colab

from google.colab import auth
auth.authenticate_user()

In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  90607      0 --:--:-- --:--:-- --:--:-- 90607
OK
55 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 55 not upgraded.
Need to get 10.8 MB of archives.
After this operation, 23.1 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.35.0_amd64.deb ...
Unpacking gcsfuse (0.35.0) ...
Setting up gcsfuse (0.35.0) ...


In [3]:
!mkdir colab_indoor
!gcsfuse indoor-data colab_indoor
# !mkdir colab_indoor/train_4
# !gcsfuse indoor-data/train_4 colab_indoor/train_4
# !mkdir colab_indoor/test_4
# !gcsfuse indoor-data/test_4 colab_indoor/test_4

2021/05/10 07:05:31.887415 Using mount point: /content/colab_indoor
2021/05/10 07:05:31.896339 Opening GCS connection...
2021/05/10 07:05:32.295832 Mounting file system "indoor-data"...
2021/05/10 07:05:32.321025 File system has been successfully mounted.


In [4]:
!ls -la -h ./colab_indoor/train_4_colcut

total 1.2G
-rw-r--r-- 1 root root  43M Apr 14 21:07 5a0546857ecc773753327266_train.csv
-rw-r--r-- 1 root root  46M Apr 14 21:07 5c3c44b80379370013e0fd2b_train.csv
-rw-r--r-- 1 root root 113M Apr 14 21:08 5d27075f03f801723c2e360f_train.csv
-rw-r--r-- 1 root root  43M Apr 14 21:08 5d27096c03f801723c31e5e0_train.csv
-rw-r--r-- 1 root root  50M Apr 14 21:08 5d27097f03f801723c320d97_train.csv
-rw-r--r-- 1 root root  12M Apr 14 21:08 5d27099f03f801723c32511d_train.csv
-rw-r--r-- 1 root root  17M Apr 14 21:08 5d2709a003f801723c3251bf_train.csv
-rw-r--r-- 1 root root  72M Apr 14 21:09 5d2709b303f801723c327472_train.csv
-rw-r--r-- 1 root root  81M Apr 14 21:09 5d2709bb03f801723c32852c_train.csv
-rw-r--r-- 1 root root  48M Apr 14 21:09 5d2709c303f801723c3299ee_train.csv
-rw-r--r-- 1 root root  48M Apr 14 21:09 5d2709d403f801723c32bd39_train.csv
-rw-r--r-- 1 root root  51M Apr 14 21:09 5d2709e003f801723c32d896_train.csv
-rw-r--r-- 1 root root 3.9M Apr 14 21:10 5da138274db8ce0c98bbd3d2_train.csv
-

In [16]:
import random
from random import sample
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter1d
from scipy.interpolate import interp1d
import time
from collections import defaultdict
import seaborn as sns

import scipy.stats as stats
from pathlib import Path
import glob
import pickle
from tqdm import tqdm
import random
import os
import copy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import multiprocessing
import math

EPOCH = 200 # default at 50
BATCH_SIZE = 64 # 256 -> learning is too slow. But theoretically we should have big batch size if we are to "pay attention" to as many records as possible
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FOLDS = 5

NUM_CORES = multiprocessing.cpu_count()
print(NUM_CORES)

IMU_OUTPUT_PATH = "./colab_indoor/train_final_preprocess/imu/"
WIFI_OUTPUT_PATH = "./colab_indoor/train_final_preprocess/wifi/"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

2


In [7]:
# train paths and test paths
train_files = sorted(glob.glob("./colab_indoor/train_4/*"))
test_files = sorted(glob.glob("./colab_indoor/test_4/*"))

# load submission file
sub_df = pd.read_csv("./colab_indoor/sample_submission.csv", index_col=0)
# sub_df[["site", "file", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
display(sub_df.head())

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,75.0,75.0


In [24]:
# Load train csv and test csv
train_df = pd.read_csv(train_files[10])
test_df = pd.read_csv(test_files[10])
display(train_df.head())
display(test_df.head())

,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,...,cc470f9a64e53fa7ef0ecb9b2caed40c9ff29d3e,2fdc216e66ab51c1ef9a06e6ffd92e4f11124dcb,42ef211be47f8c886b8e5af137c5e1369239dffb,ecdc135e3ff3143ee6afd4c9dea98f99e0d5b314,1eeea433c750d42f11799825866ad1c7db3117c5,d30bcea8da0d8f1d3b3792d13080a0705d3977ff,d61cfdb1e3d5402579286770f2327c1181fffb75,dc6c9fd26c1e983701196b6caa081bc61444816c,8838dc27035cb2f5fe9fda2d7221a1545082f2d2,89f00451d2e85eefc3ef12c27a61ff1b5d472958,9b9c1c5a535f4979f1ae544e5c9637c596bf353f,b1facdfce8c9681d2a98fb09c6bcf760338e7f73,4ad0a9ce3a396e49ca8ad7de648eaaf4952f6672,f07b56496ee3e6c6950f5a43dc2e9b013a96c200,9f8d9952638a9de993e2fdbabd273c44ac67cd97,d3898ed9c75050146f04154fc8f7323856bbb9f2,25a2a12df577debd87a56f8afafa50290ec837ed,04962969489982526983a94ffd63a8b37dcbcff4,fbb59d847d68887cfece98993bddd1af6f91feba,c867883aa74730980379fda8cb3bf7208dec9143,f64146d49e694ca0f7310849e090c24ff1f8dc20,9cbac74467908e5de62daaafd1848a2ea895f956,630afea9e645fc7d84eb18d40da63b195fefdbf7,6952e162e48482c9aef7be6c276b86c94ec826dc,03dfafd999f021000ca54f4a65b009377d76b8ef,2250b6b94c2a412765ab2c42e76418f8de76ef24,863e79d92ce227ffe323ce3d65c2fb9a5823ed21,8e9fa0b3ba590d593035fd4bbebfed1c98814285,895d551669a7e33e4c4c43bfbf7219bb15aad690,b9ef9765cfc0f6b6ee85f29775e787094972953e,99b1351f5376304f3f2ddcd3d5ba695c029611e2,cc8254a50cd31217598a4a740f760b9a4c8276e4,8b0fa9bb7d5b5ea413d2a4066d37b264e7e53132,63277c89076e93d2cf2be5fea272b4daa7c5d9d5,574198f65da59e774bf63af25270adab35246a13,c8ba7b6445629ad24ef5317a243a2cc809bd356b,f67db1935803caf91b77109c666854dd923ecf54,ae2148399209cb7f77505fb53cfcee4e93192294,25be542c6b79b46146832e481c3a4793f4143886,1c0a88fb72583c73558aaff278cbae29082663b2
0,1573789444679,1918,88.03561,109.90122,B1,-1,5dce2cf294e4900006124d2a,5d2709d403f801723c32bd39,-1.784691,3.098784,8.687047,-1.988056,2.825585,8.990739,-1.353802,1.941974,9.311101,-32.251678,-12.841980,-18.980133,18.980133,-32.343639,-12.219454,-19.652522,19.652522,-34.705268,-10.225347,-18.588266,18.588266,0.191610,-0.079466,-0.743403,0.183387,-0.063940,-0.748296,0.127559,-0.048950,-0.764860,0.063369,0.032610,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,1573789446609,3848,88.03561,109.90122,B1,-1,5dce2cf294e4900006124d2a,5d2709d403f801723c32bd39,-1.473314,1.386504,9.156513,-1.548123,1.870235,9.177041,-1.381029,1.747201,9.419000,-35.557434,-9.858353,-18.287903,18.287903,-34.794243,-10.413025,-18.727196,18.727196,-34.953874,-10.184265,-19.281826,19.281826,0.105495,-0.024781,-0.783737,0.131334,-0.038757,-0.773766,0.120622,-0.039974,-0.768472,-0.186623,-0.039108,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,1573789448539,5298,91.62208,108.70687,B1,-1,5dce2cf294e4900006124d2a,5d2709d403f801723c32bd39,-0.613656,1.092957,9.666844,-0.586432,0.929775,9.734822,-1.375224,1.577761,9.494656,-37.329849,-8.183975,-17.848511,17.848511,-37.403133,-8.179189,-17.980737,17.980737,-35.595788,-10.162183,-19.501860,19.501860,0.075151,-0.040345,-0.783424,0.064703,-0.028992,-0.785123,0.111631,-0.031414,-0.770194,-0.026357,-0.013200,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-9

,site_path_timestamp,correct_wps_ts,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,...,7f81d048aa36ef58fb8f0e767a2bead49abaf1ab,eddc077f829f5afebfad00657dadbf0091aa6ed7,c8e762424b935698e6a2022818bdfa0c0b45cd5b,387a82fb9dab2f55988bcf690d4ca99868d76e7f,813c71e4f7918fec1fa8bfc1dc5952d6e1fa6a38,7f7faeeaab023dda60f74aaa8a2cff466276e07f,70dddc9e78aa87a5906c4477b9ef0d07ab088470,376a43f7bc66e077ceb81e659641be8e3c22147b,548aef74825cf453621408fb6331819c198c6475,e17bc316c1f29f479f7aa097493045271a9038c6,d3d1e48a937c434a5ef1fea18b613f2999e8b78c,ecaee7bb517d77a8df054d171d041ad18bced9b1,174647e35a7bb2480b79f0321f4ed13074caace0,02c514fd348d943ea13807711a00f19df4206434,157119f71b167bcc847848e0769d13eef889ba93,007d6e47478bcaed8e530b8a448e80f79d19ba4d,4265b224450101fe919d2bda6dfaf57b1efa888a,b38636c45187fc59bffeb2434750ff09fffe8e4f,d681155ee335643ed2d2bfabb58c5bd237b8a638,ad877e592c5c9957e89745b5abf6ae43202cf346,96fb52ec6101b99286b825a06b8f153de352888f,fa257f727316cac58e7a3165095a64aeac175e3f,4d76b7c00e3f43619d6d912302e673194cd3109b,a2f197731084221eb08cef769b785c9dea32d0aa,77f732e3d3a1cf212d35ed98ca0a1b83092892ed,9a164ce883a1b2a53130f8bf661e80034995d961,35bd4307124edea98b58087c67df030d08240e4d,1aac60e324ba7042075222a3e91c430cbc10b0b3,c3a775d9584ddc1fd056e965e76dfa4f07ac6cac,6afc2d1143b8f19f388677a91e5bfbc25ea6a667,c001e140a76ef65d9599da868797d5ca68dd9534,6b2c8f2ac59b8b538ed7e44ad57256d42d94a2f0,4144257c211d258c312b6400f337e2a9156d2369,294a646f75591dab2bd110cbf36a763bda3547d2,b98f05cdd265c357cb492af3225dbcd4e165f198,893a2dcbe534c959b51c789fcd2e0be0da3387c1,0044a9bbcda9e331af4acffabc49da6323456a64,982ac25972916d1d0a46ef5404d4d19832f89b84,a71427c4e15bb4d047b4fe31a138e588edf3fc4d,d28452715facb93b3d1d92e829caef370ea5edca
0,5d2709d403f801723c32bd39_06882da3694b7160c0f10...,1.573706e+12,1573705854189,5859,-999.0,-999.0,-999.0,-999.0,06882da3694b7160c0f105f5,5d2709d403f801723c32bd39,-2.179963,3.023962,9.107595,-1.953401,2.734325,9.203154,-1.944560,2.792095,9.179741,14.159809,-33.941756,-15.910581,15.910581,13.648243,-33.686760,-17.055530,17.055530,13.851067,-34.652113,-17.294893,17.294893,-0.085637,0.174354,0.969787,-0.075003,0.162820,0.972324,-0.072051,0.163552,0.970261,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,5d2709d403f801723c32bd39_06882da3694b7160c0f10...,1.573706e+12,1573705854189,174,-999.0,-999.0,-999.0,-999.0,06882da3694b7160c0f105f5,5d2709d403f801723c32bd39,-2.179963,3.023962,9.107595,-1.953401,2.734325,9.203154,-1.944560,2.792095,9.179741,14.159809,-33.941756,-15.910581,15.910581,13.648243,-33.686760,-17.055530,17.055530,13.851067,-34.652113,-17.294893,17.294893,-0.085637,0.174354,0.969787,-0.075003,0.162820,0.972324,-0.072051,0.163552,0.970261,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,5d2709d403f801723c32bd39_06882da3694b7160c0f10...,1.573706e+12,1573705856155,67,-999.0,-999.0,-999.0,-999.0,06882da3694b7160c0f105f5,5d2709d403f801723c32bd39,-1.651801,2.618532,9.501848,-1.998608,2.901080,9.148263,-1.825250,2.812931,9.270919,14.411316,-31.334518,-17.805924,17.805924,15.268878,-31.605909,-16.840275,16.840275,14.019693,-32.868204,-16.345777,16.345777,-0.054039,0.155308,0.971417,-0.066823,0.164200,0.966573,-0.060922,0.162928,0.967599,...,-999,-9

---
### Processing for IMU data

---

In [26]:
# Get IMU data columns
def make_imu_data(train_path, test_path):
    # get site name
    split_path = train_path.split("/")
    file_name = split_path[-1].split("_")
    site = file_name[0]
    print("site: ", site)
    # print(IMU_OUTPUT_PATH + f"train/{site}_train.csv")

    # load df
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # filter down to imu cols
    all_cols = sorted(train_df.columns)
    imu_col_list = ["acce_", "magn_", "ahrs_", "gyro_", "rel_"]
    imu_cols = [c for c in all_cols if c.lower()[:5] in imu_col_list] # exclude magnetic fields
    imu_cols = ["x", "y", "floor_int"] + imu_cols
    imu_train_df = train_df[imu_cols]
    imu_test_df = test_df[imu_cols]

    # write the extracted imu-only files to folder
    imu_train_df.to_csv(IMU_OUTPUT_PATH + f"train/{site}_train.csv", index=False)
    imu_test_df.to_csv(IMU_OUTPUT_PATH + f"test/{site}_test.csv", index=False)

for i in range(len(train_files)):
    train_path = train_files[i]
    test_path = test_files[i]
    make_imu_data(train_path, test_path)

site:  5a0546857ecc773753327266
site:  5c3c44b80379370013e0fd2b
site:  5d27075f03f801723c2e360f
site:  5d27096c03f801723c31e5e0
site:  5d27097f03f801723c320d97
site:  5d27099f03f801723c32511d
site:  5d2709a003f801723c3251bf
site:  5d2709b303f801723c327472
site:  5d2709bb03f801723c32852c
site:  5d2709c303f801723c3299ee
site:  5d2709d403f801723c32bd39
site:  5d2709e003f801723c32d896
site:  5da138274db8ce0c98bbd3d2
site:  5da1382d4db8ce0c98bbe92e
site:  5da138314db8ce0c98bbf3a0
site:  5da138364db8ce0c98bc00f1
site:  5da1383b4db8ce0c98bc11ab
site:  5da138754db8ce0c98bca82f
site:  5da138764db8ce0c98bcaa46
site:  5da1389e4db8ce0c98bd0547
site:  5da138b74db8ce0c98bd4774
site:  5da958dd46f8266d0737457b
site:  5dbc1d84c1eb61796cf7c010
site:  5dc8cea7659e181adb076a3f


---
### Processing for DL data (Not col_cut ver)

---

In [28]:
def make_wifi_data(train_path, test_path):
    # get site name
    split_path = train_path.split("/")
    file_name = split_path[-1].split("_")
    site = file_name[0]
    print("site: ", site)
    # print(IMU_OUTPUT_PATH + f"train/{site}_train.csv")

    # load df
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # Match train and test columns
    all_train_cols = list(train_df.columns)
    all_test_cols = list(test_df.columns)
    print("all train cols: ", len(all_train_cols), "\n", "all test cols: ", len(all_test_cols))

    # get all non-overlapping columns
    no_overlap_col = list(set(all_train_cols) ^ set(all_test_cols))
    # no_overlap_col += ["floor", "file_id", "site_id"] # add other columns to exclude
    no_overlap_col += ["wifi_ts", "wps_diff", "floor", "file_id", "site_id"] # try excluding timestamp and wps_diff
    train_cols = [x for x in all_train_cols if x not in no_overlap_col]
    test_cols = [x for x in all_test_cols if x not in no_overlap_col]
    # test_cols += ["site_path_timestamp"] # test_df needs to keep "site_path_timestamp"

    # filter out the df by the columns to leave
    train_df = train_df[train_cols]
    test_df = test_df[test_cols]

    # Convert df object columns to integers and then the whole thing to tensors
    for df in [train_df, test_df]:
        obj_col = list(df.select_dtypes(include=['object']).columns)
        for col in obj_col:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].values)

    # Exclude the imu features and sort the columns
    # for picking certain columns in df
    col_count = len(test_df.columns)
    train_df_cut = train_df.iloc[:, np.r_[0:3,45:col_count]]
    test_df_cut = test_df.iloc[:, np.r_[0:3,45:col_count]]

    # sort test df columns to match train df columns
    sort_cols = sorted(train_df_cut.columns)
    sort_cols = [c for c in sort_cols if c.lower()[:4] != "magn"] # exclude magnetic fields
    sort_cols = sort_cols[:-3] # remove x, y, floor_int and then add them back into the front
    sort_cols = ["x", "y", "floor_int"] + sort_cols
    train_df = train_df_cut.reindex(columns=sort_cols)
    test_df = test_df_cut.reindex(columns=sort_cols)

    # Processing for both col_cut and all_col version
    # Replace nan values to meaningful values and then standardscale

    # Add 100 to all columns so that it becomes 1-101 rather than -100-0
    train_df.iloc[:, 3:] = train_df.iloc[:, 3:] + 101
    test_df.iloc[:, 3:] = test_df.iloc[:, 3:] + 101

    # Replace -898 with zero so that it represents no signal
    for df in [train_df, test_df]:
        all_cols = df.iloc[:, 3:].columns
        for col in all_cols:
            df[col] = np.where(df[col] == -898, 0, df[col])

    # Scale wifi feature values
    ss_cols = train_df.iloc[:, 3:].columns
    for col in ss_cols:
        ss_scaler = StandardScaler()
        train_df[col] = ss_scaler.fit_transform(train_df[col].values.reshape(-1, 1))
        test_df[col] = ss_scaler.transform(test_df[col].values.reshape(-1, 1))

    # write the extracted wifi-only files to folder
    train_df.to_csv(WIFI_OUTPUT_PATH + f"train/{site}_train.csv", index=False)
    test_df.to_csv(WIFI_OUTPUT_PATH + f"test/{site}_test.csv", index=False)

    # return train_df, test_df

In [ ]:
# # Check if it works
# train_df, test_df = make_wifi_data(train_files[1], test_files[1])
# print(len(train_df.columns))
# print(len(test_df.columns))
# print(len(train_df))
# print(len(test_df))
# print("object dtype columns in train", train_df.select_dtypes(include=['object']).columns)
# print("object dtype columns in test", test_df.select_dtypes(include=['object']).columns)
# display(train_df.head())
# display(test_df.head())

for i in range(len(train_files)):
    train_path = train_files[i]
    test_path = test_files[i]
    make_wifi_data(train_path, test_path)

site:  5a0546857ecc773753327266
all train cols:  3456 
 all test cols:  2486


In [ ]:
# # Match train and test columns
# all_train_cols = list(train_df.columns)
# all_test_cols = list(test_df.columns)
# print("all train cols: ", len(all_train_cols), "\n", "all test cols: ", len(all_test_cols))

# # get all non-overlapping columns
# no_overlap_col = list(set(all_train_cols) ^ set(all_test_cols))
# # no_overlap_col += ["floor", "file_id", "site_id"] # add other columns to exclude
# no_overlap_col += ["wifi_ts", "wps_diff", "floor", "file_id", "site_id"] # try excluding timestamp and wps_diff
# train_cols = [x for x in all_train_cols if x not in no_overlap_col]
# test_cols = [x for x in all_test_cols if x not in no_overlap_col]
# # test_cols += ["site_path_timestamp"] # test_df needs to keep "site_path_timestamp"

# # filter out the df by the columns to leave
# train_df = train_df[train_cols]
# test_df = test_df[test_cols]

# # # Drop some columns not necessary as a feature
# # drop_cols = ["wifi_ts", "floor", "file_id", "site_id"]
# # for df in [train_df, test_df]:
# #     df = df.drop(columns=drop_cols, inplace=True)

# # Convert df object columns to integers and then the whole thing to tensors
# for df in [train_df, test_df]:
#     obj_col = list(df.select_dtypes(include=['object']).columns)
#     for col in obj_col:
#         le = LabelEncoder()
#         df[col] = le.fit_transform(df[col].values)

# print(len(train_df.columns))
# print(len(test_df.columns))
# print(len(train_df))
# print(len(test_df))
# print("object dtype columns in train", train_df.select_dtypes(include=['object']).columns)
# print("object dtype columns in test", test_df.select_dtypes(include=['object']).columns)
# display(train_df.head())
# display(test_df.head())

all train cols:  2198 
 all test cols:  1827
1655
1655
10027
1223
object dtype columns in train Index([], dtype='object')
object dtype columns in test Index([], dtype='object')


,x,y,floor_int,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,gyro_1000_z_avg,gyro_2000_x_avg,gyro_2000_y_avg,gyro_2000_z_avg,gyro_5000_x_avg,...,6280c1144ca0ab506236c4abb8fd875e072d72d9,514a2f7e149d781a35461d96c166bae94a5dc360,c0872f1eed9124b5685e5f72ad4399277cefe617,3313aa8afd06c98fe21615ef6254e6e7f2b7f28e,929f1c42922a97db0d0b73cc0abcdc152c54c405,8afe5ad8ef15447c16f3ca589063a231409ada8c,8db0aabc861356d9e18b8a6d1d33716895bbfbee,2472ffc3d02d46fab1c4a8a457147e2715da4e6f,9ed7e1a8713bfb9514730e609b5b7604957bdfca,a2210cb2461c8a9879ed37da7d0642b6540faf82,83a5aaa78fdac5c94dfc802d2f513c94934c246c,26cfaf5221bbe23b7ee44ca3f81cc33df5c5cdda,e99cf62efbb6161bc72c592fb247d235a312afee,9d1c203e1d50645ba429a917d3fc810990204a2b,f39b070cb61a4feefeafb3a5c6facf3287d42c70,379e051967013800aff1a4eec89f9b74f28a79d6,576543c3c23a960b02b3d819c6f9c8a5a5049c4e,1a0e770894a9fed61e7453babd6215f7104ef451,38d0b981e7d64d7ec1472c3a3f3a29914f7ac323,930a87a62e23ce97b65912d0ab51fa0a8b138826,ce735431fd365f14779344a5c5a38ccee53aa820,ecdc135e3ff3143ee6afd4c9dea98f99e0d5b314,1eeea433c750d42f11799825866ad1c7db3117c5,dc6c9fd26c1e983701196b6caa081bc61444816c,8838dc27035cb2f5fe9fda2d7221a1545082f2d2,89f00451d2e85eefc3ef12c27a61ff1b5d472958,9b9c1c5a535f4979f1ae544e5c9637c596bf353f,b1facdfce8c9681d2a98fb09c6bcf760338e7f73,4ad0a9ce3a396e49ca8ad7de648eaaf4952f6672,f07b56496ee3e6c6950f5a43dc2e9b013a96c200,d3898ed9c75050146f04154fc8f7323856bbb9f2,fbb59d847d68887cfece98993bddd1af6f91feba,c867883aa74730980379fda8cb3bf7208dec9143,9cbac74467908e5de62daaafd1848a2ea895f956,630afea9e645fc7d84eb18d40da63b195fefdbf7,2250b6b94c2a412765ab2c42e76418f8de76ef24,863e79d92ce227ffe323ce3d65c2fb9a5823ed21,8e9fa0b3ba590d593035fd4bbebfed1c98814285,8b0fa9bb7d5b5ea413d2a4066d37b264e7e53132,25be542c6b79b46146832e481c3a4793f4143886
0,88.03561,109.90122,-1,-1.784691,3.098784,8.687047,-1.988056,2.825585,8.990739,-1.353802,1.941974,9.311101,-32.251678,-12.841980,-18.980133,18.980133,-32.343639,-12.219454,-19.652522,19.652522,-34.705268,-10.225347,-18.588266,18.588266,0.191610,-0.079466,-0.743403,0.183387,-0.063940,-0.748296,0.127559,-0.048950,-0.764860,0.063369,0.032610,-0.138905,-0.148707,-0.000721,0.018886,-0.066312,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,88.03561,109.90122,-1,-1.473314,1.386504,9.156513,-1.548123,1.870235,9.177041,-1.381029,1.747201,9.419000,-35.557434,-9.858353,-18.287903,18.287903,-34.794243,-10.413025,-18.727196,18.727196,-34.953874,-10.184265,-19.281826,19.281826,0.105495,-0.024781,-0.783737,0.131334,-0.038757,-0.773766,0.120622,-0.039974,-0.768472,-0.186623,-0.039108,-0.043037,-0.074626,-0.067391,0.007595,-0.056719,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,91.62208,108.70687,-1,-0.613656,1.092957,9.666844,-0.586432,0.929775,9.734822,-1.375224,1.577761,9.494656,-37.329849,-8.183975,-17.848511,17.848511,-37.403133,-8.179189,-17.980737,17.980737,-35.595788,-10.162183,-19.501860,19.501860,0.075151,-0.040345,-0.783424,0.064703,-0.028992,-0.785123,0.111631,-0.031414,-0.770194,-0.026357,-0.013200,0.039639,-0.025248,0.023123,-0.026966,-0.021917,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-

,x,y,floor_int,acce_1000_x_avg,acce_1000_y_avg,acce_1000_z_avg,acce_2000_x_avg,acce_2000_y_avg,acce_2000_z_avg,acce_5000_x_avg,acce_5000_y_avg,acce_5000_z_avg,magn_1000_x_avg,magn_1000_y_avg,magn_1000_z_avg,magn_1000_st_avg,magn_2000_x_avg,magn_2000_y_avg,magn_2000_z_avg,magn_2000_st_avg,magn_5000_x_avg,magn_5000_y_avg,magn_5000_z_avg,magn_5000_st_avg,ahrs_1000_x_avg,ahrs_1000_y_avg,ahrs_1000_z_avg,ahrs_2000_x_avg,ahrs_2000_y_avg,ahrs_2000_z_avg,ahrs_5000_x_avg,ahrs_5000_y_avg,ahrs_5000_z_avg,gyro_1000_x_avg,gyro_1000_y_avg,gyro_1000_z_avg,gyro_2000_x_avg,gyro_2000_y_avg,gyro_2000_z_avg,gyro_5000_x_avg,...,589984148160aacd702584f0fef65ef296ee7998,7a3fb52b99d25506e1481bb80c72e440f1e79fe4,562899f5a8998c2b6f71b0a7e32543530a749627,826c100c5853efceb5defbecc63e718b88d1aeab,630afea9e645fc7d84eb18d40da63b195fefdbf7,12f1df34035ca4d7161ef3af72ab7a79b7729c6c,964814d9a647d868848b51a61919e13483cd38ef,1248845704d76c9eba53087337da21bd551b7861,24fb865d5b9f504500b9f69aa6086b4d9c361bf6,7c627a30cee4980197aa525e54639e1ea0d4f9e0,e35e169b39cc4e06bf3c795e0a8258fd43381075,e59324c46c7a9086b479d69d017979f0dd45fac3,88078f208a2db1422f5e07503bb18030cee0e3ce,772b8e93f8ee637000df911cdc2c82160f57791f,bc2a377f005f9057e2779374672f2679012683f7,c80dfed1ac5d2eb09533847fed6e9f3e333ddc7a,eb524e65df31558d801c637fd04d00177b668cfa,7f81d048aa36ef58fb8f0e767a2bead49abaf1ab,387a82fb9dab2f55988bcf690d4ca99868d76e7f,813c71e4f7918fec1fa8bfc1dc5952d6e1fa6a38,7f7faeeaab023dda60f74aaa8a2cff466276e07f,e17bc316c1f29f479f7aa097493045271a9038c6,d3d1e48a937c434a5ef1fea18b613f2999e8b78c,02c514fd348d943ea13807711a00f19df4206434,4265b224450101fe919d2bda6dfaf57b1efa888a,b38636c45187fc59bffeb2434750ff09fffe8e4f,d681155ee335643ed2d2bfabb58c5bd237b8a638,ad877e592c5c9957e89745b5abf6ae43202cf346,96fb52ec6101b99286b825a06b8f153de352888f,77f732e3d3a1cf212d35ed98ca0a1b83092892ed,9a164ce883a1b2a53130f8bf661e80034995d961,35bd4307124edea98b58087c67df030d08240e4d,c3a775d9584ddc1fd056e965e76dfa4f07ac6cac,6afc2d1143b8f19f388677a91e5bfbc25ea6a667,4144257c211d258c312b6400f337e2a9156d2369,294a646f75591dab2bd110cbf36a763bda3547d2,b98f05cdd265c357cb492af3225dbcd4e165f198,893a2dcbe534c959b51c789fcd2e0be0da3387c1,982ac25972916d1d0a46ef5404d4d19832f89b84,d28452715facb93b3d1d92e829caef370ea5edca
0,-999.0,-999.0,-999.0,-2.179963,3.023962,9.107595,-1.953401,2.734325,9.203154,-1.944560,2.792095,9.179741,14.159809,-33.941756,-15.910581,15.910581,13.648243,-33.686760,-17.055530,17.055530,13.851067,-34.652113,-17.294893,17.294893,-0.085637,0.174354,0.969787,-0.075003,0.162820,0.972324,-0.072051,0.163552,0.970261,-0.082571,-0.038393,-0.058174,-0.028216,0.015426,-0.011096,-0.003186,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,-999.0,-999.0,-999.0,-2.179963,3.023962,9.107595,-1.953401,2.734325,9.203154,-1.944560,2.792095,9.179741,14.159809,-33.941756,-15.910581,15.910581,13.648243,-33.686760,-17.055530,17.055530,13.851067,-34.652113,-17.294893,17.294893,-0.085637,0.174354,0.969787,-0.075003,0.162820,0.972324,-0.072051,0.163552,0.970261,-0.082571,-0.038393,-0.058174,-0.028216,0.015426,-0.011096,-0.003186,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,-999.0,-999.0,-999.0,-1.651801,2.618532,9.501848,-1.998608,2.901080,9.148263,-1.825250,2.812931,9.270919,14.411316,-31.334518,-17.805924,17.805924,15.268878,-31.605909,-16.840275,16.840275,14.019693,-32.868204,-16.345777,16.345777,-0.054039,0.155308,0.971417,-0.066823,0.164200,0.966573,-0.060922,0.162928,0.967599,0.041121,-0.065208,-0.098741,-0.042632,0.011663,0.009059,-0.021100,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-99

In [ ]:
# # Exclude the imu features and sort the columns

# # for picking certain columns in df
# # https://stackoverflow.com/questions/41256648/select-multiple-ranges-of-columns-in-pandas-dataframe
# col_count = len(test_df.columns)
# train_df_cut = train_df.iloc[:, np.r_[0:3,45:col_count]]
# test_df_cut = test_df.iloc[:, np.r_[0:3,45:col_count]]

# # sort test df columns to match train df columns
# sort_cols = sorted(train_df_cut.columns)
# sort_cols = [c for c in sort_cols if c.lower()[:4] != "magn"] # exclude magnetic fields
# sort_cols = sort_cols[:-3] # remove x, y, floor_int and then add them back into the front
# sort_cols = ["x", "y", "floor_int"] + sort_cols
# train_df = train_df_cut.reindex(columns=sort_cols)
# test_df = test_df_cut.reindex(columns=sort_cols)

# display(train_df.head())
# display(test_df.head())

,x,y,floor_int,000466c4e21420389c450f9d5fd40d4df77384f1,00f494ec6739cf9454c175f24c5dffffd424da06,0129a8f5d4e3dac54960345b7af3e04522a426db,017f9b5cdcf09f514ca7abb944898d4fdb7ae26c,018a066872b67d5b33570ddeb6142c602a22e451,018f780ecb43d52964d31459005b3afa8720398c,01c42aebb61b39fe47a7d01b7725dcafb66e91be,01e6e57a288870e95601691f13f78dcdb1674ac1,020eeac348fda29f055abc26ee08acde97b2a791,0255bd572556fa6fda45b88e7a1aeef6e4a3be87,02ac9eba5ebfa2a124c2221f2595f0dc9eed5cb6,02c514fd348d943ea13807711a00f19df4206434,02e3cf652b4fb79bfd1117278046aba39c7c9f77,030e8b570cb7b90a00ec69f9d7f509bad0e3e32f,033356b8711c191c4f5bf4d0290925c5346bda50,033a638b4bf227f1f3fb9ca67e6dfc646570aa5e,03618bce45adf0c5a9d92bf5377250de22087a14,036bd3f5aecb8252717e88099a7d425c710a9b3b,03de6c22500cae518256efa2d67ccb806adfe95d,03e33221e656092f561aadc53bb80b4c043040f0,04662b700fd39a057e28d86378d8a21fa1b07c96,048af33bf707077e05ea4877ccb1bee484ed182d,049fba985a3be2563661e2b552e324054190c38a,04fbb0681cee7aafabb1d85eb06c92ec33514629,050d22393064308792e23dfd5605049d69555523,0511d9b3a713b7174338d88bfcf2c4d6feccd69c,051facc6a84b8dde709e11721a4a499ad8bedfd0,059168d343192c492bdd162c9d09bbe8080b169e,05e36651a5c67ecb30c6e46c8404a4f2777ced8b,06028150b43acb7e11121888ab434cdd5f679f1d,0606aeb26c0349fada4ba49d7f0d931c97c0e76a,061ac2c5d1a9c1d28bd404200b24df8e31f9f0f4,062c394a0e75d8b7163ca7d504cfd1d6acaa4edf,06496b72ca7ff23b5c208db5aee6b084a7cc4382,067e3197ca5695e30cda75131da9e2fb4b36ab59,0705f3e1383ca1c28157be5ccce87e030d542c5d,0708b2d25c6d095365b69227447f148bd2b1c9cc,...,faf431ff8bb7aa6ff52385c887c77e01d83f0d0f,fb35b1e712ba64fa07a7ef102e33cec631e6585a,fb4b9d8f69df00728d75297ed1222fe243367d89,fb80310b778ea3476d9f16eb706ace9953d22e75,fba76c36f912d5c059f9685bbdcb452a97cac8f0,fbb59d847d68887cfece98993bddd1af6f91feba,fbe1508eca4a959d57da0fba84a912a709d9be8e,fc017c9c685f0dbcf1dc1fb382facb5658b91fd7,fc032ede7da4a5dc16d795e3313bb8b4582e3e8d,fc0507743e39ce0561e1599f841375400b813517,fc241e8874f0fd25b312bc69938cd02bfd5f9fb1,fc381bfc93afe1c7a3e052c7cabbb445ffea9f12,fc520c5f525e17e52fe05b766251c86705469e5e,fc87b98884fc1a313d5d4d7c4e2d2793b7f35f85,fc88f6a4f0351f8a2b34d256fcda35579d9e0ead,fc8fd87104388e513937ac13696b6f72af2add53,fca736127d9fdd84726c628a827390b8ef2b7115,fcc6ae39eedac1187206df50b6d75837673c8758,fceeacbea051939cab5155ad7319e8b0e36acff7,fcefc5b8b1702f2e07b62d977d9f402eaac65eb4,fcf0cd88c10fd04341d7163f2e0b9722f1992ec0,fcfffc33405e558cc135986605215c4f0a3dd602,fd10bc4364f9a03a71db349c0f2b5e981c12f94b,fd2cafb052553e89f30d043a2101c4ab20f0a2d1,fd333b9e73ff5c3767c9b746010d5b4c78b16d07,fd9680adc77a7fa3ab82f101f4c5058824353720,fdcac7c1f1fc209996fb53ad9a5510b6c6281a84,fdcfe3b6adc8a422a76d7e704df41e2a073ebeaa,fdfbb5cd1a558a9a8fcf9484d7f6ab9c7be324cf,fe597fb70e5347917531fda7cdb1a25f5fa28cf5,fe721a2d2e568ba8c4e7649011e426a218d656fe,feedbdf38cbcd80d987f4911009a9178c94a14b4,ff141398480b0fd43ee047889d6a1a0a2465bc43,ff435c869f9353d4b7ce39a7b979503e83b6f16d,ff67be9486f55388fbea254c55233b72a8646e3a,ff68055a0a2beea10a265bcc4d40242e0cc4af97,ffb5692c0c58a68fc528f5db8272db7375546fc8,ffd040ff4318abbb0b69a875ddc97826ed71259d,fff62d53bd41aeb9e51e9d792536da349e1d8828,fffcccea4ac3efd9b346e5f55671fa4b7ec6088a
0,88.03561,109.90122,-1,-999,-999,-999,-999,-73,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-77,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-81,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-56,-999,-999
1,88.03561,109.90122,-1,-999,-999,-999,-999,-78,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-77,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-81,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-56,-999,-999
2,91.6

,x,y,floor_int,000466c4e21420389c450f9d5fd40d4df77384f1,00f494ec6739cf9454c175f24c5dffffd424da06,0129a8f5d4e3dac54960345b7af3e04522a426db,017f9b5cdcf09f514ca7abb944898d4fdb7ae26c,018a066872b67d5b33570ddeb6142c602a22e451,018f780ecb43d52964d31459005b3afa8720398c,01c42aebb61b39fe47a7d01b7725dcafb66e91be,01e6e57a288870e95601691f13f78dcdb1674ac1,020eeac348fda29f055abc26ee08acde97b2a791,0255bd572556fa6fda45b88e7a1aeef6e4a3be87,02ac9eba5ebfa2a124c2221f2595f0dc9eed5cb6,02c514fd348d943ea13807711a00f19df4206434,02e3cf652b4fb79bfd1117278046aba39c7c9f77,030e8b570cb7b90a00ec69f9d7f509bad0e3e32f,033356b8711c191c4f5bf4d0290925c5346bda50,033a638b4bf227f1f3fb9ca67e6dfc646570aa5e,03618bce45adf0c5a9d92bf5377250de22087a14,036bd3f5aecb8252717e88099a7d425c710a9b3b,03de6c22500cae518256efa2d67ccb806adfe95d,03e33221e656092f561aadc53bb80b4c043040f0,04662b700fd39a057e28d86378d8a21fa1b07c96,048af33bf707077e05ea4877ccb1bee484ed182d,049fba985a3be2563661e2b552e324054190c38a,04fbb0681cee7aafabb1d85eb06c92ec33514629,050d22393064308792e23dfd5605049d69555523,0511d9b3a713b7174338d88bfcf2c4d6feccd69c,051facc6a84b8dde709e11721a4a499ad8bedfd0,059168d343192c492bdd162c9d09bbe8080b169e,05e36651a5c67ecb30c6e46c8404a4f2777ced8b,06028150b43acb7e11121888ab434cdd5f679f1d,0606aeb26c0349fada4ba49d7f0d931c97c0e76a,061ac2c5d1a9c1d28bd404200b24df8e31f9f0f4,062c394a0e75d8b7163ca7d504cfd1d6acaa4edf,06496b72ca7ff23b5c208db5aee6b084a7cc4382,067e3197ca5695e30cda75131da9e2fb4b36ab59,0705f3e1383ca1c28157be5ccce87e030d542c5d,0708b2d25c6d095365b69227447f148bd2b1c9cc,...,faf431ff8bb7aa6ff52385c887c77e01d83f0d0f,fb35b1e712ba64fa07a7ef102e33cec631e6585a,fb4b9d8f69df00728d75297ed1222fe243367d89,fb80310b778ea3476d9f16eb706ace9953d22e75,fba76c36f912d5c059f9685bbdcb452a97cac8f0,fbb59d847d68887cfece98993bddd1af6f91feba,fbe1508eca4a959d57da0fba84a912a709d9be8e,fc017c9c685f0dbcf1dc1fb382facb5658b91fd7,fc032ede7da4a5dc16d795e3313bb8b4582e3e8d,fc0507743e39ce0561e1599f841375400b813517,fc241e8874f0fd25b312bc69938cd02bfd5f9fb1,fc381bfc93afe1c7a3e052c7cabbb445ffea9f12,fc520c5f525e17e52fe05b766251c86705469e5e,fc87b98884fc1a313d5d4d7c4e2d2793b7f35f85,fc88f6a4f0351f8a2b34d256fcda35579d9e0ead,fc8fd87104388e513937ac13696b6f72af2add53,fca736127d9fdd84726c628a827390b8ef2b7115,fcc6ae39eedac1187206df50b6d75837673c8758,fceeacbea051939cab5155ad7319e8b0e36acff7,fcefc5b8b1702f2e07b62d977d9f402eaac65eb4,fcf0cd88c10fd04341d7163f2e0b9722f1992ec0,fcfffc33405e558cc135986605215c4f0a3dd602,fd10bc4364f9a03a71db349c0f2b5e981c12f94b,fd2cafb052553e89f30d043a2101c4ab20f0a2d1,fd333b9e73ff5c3767c9b746010d5b4c78b16d07,fd9680adc77a7fa3ab82f101f4c5058824353720,fdcac7c1f1fc209996fb53ad9a5510b6c6281a84,fdcfe3b6adc8a422a76d7e704df41e2a073ebeaa,fdfbb5cd1a558a9a8fcf9484d7f6ab9c7be324cf,fe597fb70e5347917531fda7cdb1a25f5fa28cf5,fe721a2d2e568ba8c4e7649011e426a218d656fe,feedbdf38cbcd80d987f4911009a9178c94a14b4,ff141398480b0fd43ee047889d6a1a0a2465bc43,ff435c869f9353d4b7ce39a7b979503e83b6f16d,ff67be9486f55388fbea254c55233b72a8646e3a,ff68055a0a2beea10a265bcc4d40242e0cc4af97,ffb5692c0c58a68fc528f5db8272db7375546fc8,ffd040ff4318abbb0b69a875ddc97826ed71259d,fff62d53bd41aeb9e51e9d792536da349e1d8828,fffcccea4ac3efd9b346e5f55671fa4b7ec6088a
0,-999.0,-999.0,-999.0,-70,-999,-999,-999,-999,-999,-78,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-76,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-75,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-86,-999,-999,-999
1,-999.0,-999.0,-999.0,-70,-999,-999,-999,-999,-999,-78,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-76,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-75,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-86,-999,-999,-999
2,-999.0,-

In [ ]:
# # Processing for both col_cut and all_col version
# # Replace nan values to meaningful values and then standardscale

# # Add 100 to all columns so that it becomes 1-101 rather than -100-0
# train_df.iloc[:, 3:] = train_df.iloc[:, 3:] + 101
# test_df.iloc[:, 3:] = test_df.iloc[:, 3:] + 101

# # Replace -898 with zero so that it represents no signal
# for df in [train_df, test_df]:
#     all_cols = df.iloc[:, 3:].columns
#     for col in all_cols:
#         df[col] = np.where(df[col] == -898, 0, df[col])

# # Try without using scalers for now. 
# # Apply MinMaxScaler to each IMU columns
# # current_cols = list(train_df.columns)
# # imu_cols = current_cols[:23]
# # exception_columns = ["wps_diff", "x", "y", "floor_int", "rel_diff", "rel_x", "rel_y"]
# # imu_cols = [x for x in imu_cols if x not in exception_columns]
# # print("imu_cols: ", imu_cols)
# ss_cols = train_df.iloc[:, 3:].columns
# for col in ss_cols:
#     ss_scaler = StandardScaler()
#     train_df[col] = ss_scaler.fit_transform(train_df[col].values.reshape(-1, 1))
#     test_df[col] = ss_scaler.transform(test_df[col].values.reshape(-1, 1))

# display(train_df.head())
# display(test_df.head())

,x,y,floor_int,000466c4e21420389c450f9d5fd40d4df77384f1,00f494ec6739cf9454c175f24c5dffffd424da06,0129a8f5d4e3dac54960345b7af3e04522a426db,017f9b5cdcf09f514ca7abb944898d4fdb7ae26c,018a066872b67d5b33570ddeb6142c602a22e451,018f780ecb43d52964d31459005b3afa8720398c,01c42aebb61b39fe47a7d01b7725dcafb66e91be,01e6e57a288870e95601691f13f78dcdb1674ac1,020eeac348fda29f055abc26ee08acde97b2a791,0255bd572556fa6fda45b88e7a1aeef6e4a3be87,02ac9eba5ebfa2a124c2221f2595f0dc9eed5cb6,02c514fd348d943ea13807711a00f19df4206434,02e3cf652b4fb79bfd1117278046aba39c7c9f77,030e8b570cb7b90a00ec69f9d7f509bad0e3e32f,033356b8711c191c4f5bf4d0290925c5346bda50,033a638b4bf227f1f3fb9ca67e6dfc646570aa5e,03618bce45adf0c5a9d92bf5377250de22087a14,036bd3f5aecb8252717e88099a7d425c710a9b3b,03de6c22500cae518256efa2d67ccb806adfe95d,03e33221e656092f561aadc53bb80b4c043040f0,04662b700fd39a057e28d86378d8a21fa1b07c96,048af33bf707077e05ea4877ccb1bee484ed182d,049fba985a3be2563661e2b552e324054190c38a,04fbb0681cee7aafabb1d85eb06c92ec33514629,050d22393064308792e23dfd5605049d69555523,0511d9b3a713b7174338d88bfcf2c4d6feccd69c,051facc6a84b8dde709e11721a4a499ad8bedfd0,059168d343192c492bdd162c9d09bbe8080b169e,05e36651a5c67ecb30c6e46c8404a4f2777ced8b,06028150b43acb7e11121888ab434cdd5f679f1d,0606aeb26c0349fada4ba49d7f0d931c97c0e76a,061ac2c5d1a9c1d28bd404200b24df8e31f9f0f4,062c394a0e75d8b7163ca7d504cfd1d6acaa4edf,06496b72ca7ff23b5c208db5aee6b084a7cc4382,067e3197ca5695e30cda75131da9e2fb4b36ab59,0705f3e1383ca1c28157be5ccce87e030d542c5d,0708b2d25c6d095365b69227447f148bd2b1c9cc,...,faf431ff8bb7aa6ff52385c887c77e01d83f0d0f,fb35b1e712ba64fa07a7ef102e33cec631e6585a,fb4b9d8f69df00728d75297ed1222fe243367d89,fb80310b778ea3476d9f16eb706ace9953d22e75,fba76c36f912d5c059f9685bbdcb452a97cac8f0,fbb59d847d68887cfece98993bddd1af6f91feba,fbe1508eca4a959d57da0fba84a912a709d9be8e,fc017c9c685f0dbcf1dc1fb382facb5658b91fd7,fc032ede7da4a5dc16d795e3313bb8b4582e3e8d,fc0507743e39ce0561e1599f841375400b813517,fc241e8874f0fd25b312bc69938cd02bfd5f9fb1,fc381bfc93afe1c7a3e052c7cabbb445ffea9f12,fc520c5f525e17e52fe05b766251c86705469e5e,fc87b98884fc1a313d5d4d7c4e2d2793b7f35f85,fc88f6a4f0351f8a2b34d256fcda35579d9e0ead,fc8fd87104388e513937ac13696b6f72af2add53,fca736127d9fdd84726c628a827390b8ef2b7115,fcc6ae39eedac1187206df50b6d75837673c8758,fceeacbea051939cab5155ad7319e8b0e36acff7,fcefc5b8b1702f2e07b62d977d9f402eaac65eb4,fcf0cd88c10fd04341d7163f2e0b9722f1992ec0,fcfffc33405e558cc135986605215c4f0a3dd602,fd10bc4364f9a03a71db349c0f2b5e981c12f94b,fd2cafb052553e89f30d043a2101c4ab20f0a2d1,fd333b9e73ff5c3767c9b746010d5b4c78b16d07,fd9680adc77a7fa3ab82f101f4c5058824353720,fdcac7c1f1fc209996fb53ad9a5510b6c6281a84,fdcfe3b6adc8a422a76d7e704df41e2a073ebeaa,fdfbb5cd1a558a9a8fcf9484d7f6ab9c7be324cf,fe597fb70e5347917531fda7cdb1a25f5fa28cf5,fe721a2d2e568ba8c4e7649011e426a218d656fe,feedbdf38cbcd80d987f4911009a9178c94a14b4,ff141398480b0fd43ee047889d6a1a0a2465bc43,ff435c869f9353d4b7ce39a7b979503e83b6f16d,ff67be9486f55388fbea254c55233b72a8646e3a,ff68055a0a2beea10a265bcc4d40242e0cc4af97,ffb5692c0c58a68fc528f5db8272db7375546fc8,ffd040ff4318abbb0b69a875ddc97826ed71259d,fff62d53bd41aeb9e51e9d792536da349e1d8828,fffcccea4ac3efd9b346e5f55671fa4b7ec6088a
0,88.03561,109.90122,-1,-0.184996,-0.35749,-0.088856,-0.194877,1.652452,-0.099524,-0.03603,-0.128593,-0.538429,-0.301676,-0.073751,-0.081487,-0.152991,-0.419914,-0.075193,-0.078753,-0.126978,-0.408125,3.445257,-0.081746,-0.04574,-0.372555,-0.227612,-0.321157,-0.038707,-0.167135,-0.096832,-0.14491,-0.280098,-0.12043,-0.122216,-0.283828,-0.089657,-0.0173,-0.128285,-0.291893,-0.229866,...,-0.047512,-0.418097,-0.22137,-0.513919,-0.372022,-0.024469,2.345192,-0.039477,-0.073593,-0.181996,-0.135551,-0.063062,-0.031541,-0.038707,-0.115313,-0.120531,-0.091079,-0.052695,-0.306357,-0.042133,-0.03157,-0.048851,-0.052868,-0.072052,-0.082584,-0.121637,-0.130545,-0.539048,-0.283626,-0.009987,-0.059156,-0.21345,-0.1087,-0.404672,-0.290144,-0.286308,-0.546898,4.521277,-0.068358,-0.070661
1,88.03561,109.90122,-1,-0.184996,-0.35749

,x,y,floor_int,000466c4e21420389c450f9d5fd40d4df77384f1,00f494ec6739cf9454c175f24c5dffffd424da06,0129a8f5d4e3dac54960345b7af3e04522a426db,017f9b5cdcf09f514ca7abb944898d4fdb7ae26c,018a066872b67d5b33570ddeb6142c602a22e451,018f780ecb43d52964d31459005b3afa8720398c,01c42aebb61b39fe47a7d01b7725dcafb66e91be,01e6e57a288870e95601691f13f78dcdb1674ac1,020eeac348fda29f055abc26ee08acde97b2a791,0255bd572556fa6fda45b88e7a1aeef6e4a3be87,02ac9eba5ebfa2a124c2221f2595f0dc9eed5cb6,02c514fd348d943ea13807711a00f19df4206434,02e3cf652b4fb79bfd1117278046aba39c7c9f77,030e8b570cb7b90a00ec69f9d7f509bad0e3e32f,033356b8711c191c4f5bf4d0290925c5346bda50,033a638b4bf227f1f3fb9ca67e6dfc646570aa5e,03618bce45adf0c5a9d92bf5377250de22087a14,036bd3f5aecb8252717e88099a7d425c710a9b3b,03de6c22500cae518256efa2d67ccb806adfe95d,03e33221e656092f561aadc53bb80b4c043040f0,04662b700fd39a057e28d86378d8a21fa1b07c96,048af33bf707077e05ea4877ccb1bee484ed182d,049fba985a3be2563661e2b552e324054190c38a,04fbb0681cee7aafabb1d85eb06c92ec33514629,050d22393064308792e23dfd5605049d69555523,0511d9b3a713b7174338d88bfcf2c4d6feccd69c,051facc6a84b8dde709e11721a4a499ad8bedfd0,059168d343192c492bdd162c9d09bbe8080b169e,05e36651a5c67ecb30c6e46c8404a4f2777ced8b,06028150b43acb7e11121888ab434cdd5f679f1d,0606aeb26c0349fada4ba49d7f0d931c97c0e76a,061ac2c5d1a9c1d28bd404200b24df8e31f9f0f4,062c394a0e75d8b7163ca7d504cfd1d6acaa4edf,06496b72ca7ff23b5c208db5aee6b084a7cc4382,067e3197ca5695e30cda75131da9e2fb4b36ab59,0705f3e1383ca1c28157be5ccce87e030d542c5d,0708b2d25c6d095365b69227447f148bd2b1c9cc,...,faf431ff8bb7aa6ff52385c887c77e01d83f0d0f,fb35b1e712ba64fa07a7ef102e33cec631e6585a,fb4b9d8f69df00728d75297ed1222fe243367d89,fb80310b778ea3476d9f16eb706ace9953d22e75,fba76c36f912d5c059f9685bbdcb452a97cac8f0,fbb59d847d68887cfece98993bddd1af6f91feba,fbe1508eca4a959d57da0fba84a912a709d9be8e,fc017c9c685f0dbcf1dc1fb382facb5658b91fd7,fc032ede7da4a5dc16d795e3313bb8b4582e3e8d,fc0507743e39ce0561e1599f841375400b813517,fc241e8874f0fd25b312bc69938cd02bfd5f9fb1,fc381bfc93afe1c7a3e052c7cabbb445ffea9f12,fc520c5f525e17e52fe05b766251c86705469e5e,fc87b98884fc1a313d5d4d7c4e2d2793b7f35f85,fc88f6a4f0351f8a2b34d256fcda35579d9e0ead,fc8fd87104388e513937ac13696b6f72af2add53,fca736127d9fdd84726c628a827390b8ef2b7115,fcc6ae39eedac1187206df50b6d75837673c8758,fceeacbea051939cab5155ad7319e8b0e36acff7,fcefc5b8b1702f2e07b62d977d9f402eaac65eb4,fcf0cd88c10fd04341d7163f2e0b9722f1992ec0,fcfffc33405e558cc135986605215c4f0a3dd602,fd10bc4364f9a03a71db349c0f2b5e981c12f94b,fd2cafb052553e89f30d043a2101c4ab20f0a2d1,fd333b9e73ff5c3767c9b746010d5b4c78b16d07,fd9680adc77a7fa3ab82f101f4c5058824353720,fdcac7c1f1fc209996fb53ad9a5510b6c6281a84,fdcfe3b6adc8a422a76d7e704df41e2a073ebeaa,fdfbb5cd1a558a9a8fcf9484d7f6ab9c7be324cf,fe597fb70e5347917531fda7cdb1a25f5fa28cf5,fe721a2d2e568ba8c4e7649011e426a218d656fe,feedbdf38cbcd80d987f4911009a9178c94a14b4,ff141398480b0fd43ee047889d6a1a0a2465bc43,ff435c869f9353d4b7ce39a7b979503e83b6f16d,ff67be9486f55388fbea254c55233b72a8646e3a,ff68055a0a2beea10a265bcc4d40242e0cc4af97,ffb5692c0c58a68fc528f5db8272db7375546fc8,ffd040ff4318abbb0b69a875ddc97826ed71259d,fff62d53bd41aeb9e51e9d792536da349e1d8828,fffcccea4ac3efd9b346e5f55671fa4b7ec6088a
0,-999.0,-999.0,-999.0,7.510893,-0.35749,-0.088856,-0.194877,-0.656789,-0.099524,33.605038,-0.128593,-0.538429,-0.301676,-0.073751,-0.081487,-0.152991,-0.419914,-0.075193,-0.078753,-0.126978,-0.408125,-0.242163,-0.081746,-0.04574,-0.372555,-0.227612,-0.321157,-0.038707,-0.167135,-0.096832,-0.14491,-0.280098,-0.12043,-0.122216,2.565764,-0.089657,-0.0173,-0.128285,-0.291893,-0.229866,...,-0.047512,-0.418097,-0.22137,-0.513919,-0.372022,-0.024469,-0.273986,-0.039477,13.185440,-0.181996,-0.135551,-0.063062,-0.031541,-0.038707,-0.115313,-0.120531,-0.091079,-0.052695,-0.306357,-0.042133,-0.03157,-0.048851,-0.052868,-0.072052,-0.082584,-0.121637,-0.130545,-0.539048,-0.283626,-0.009987,-0.059156,-0.21345,-0.1087,-0.404672,-0.290144,-0.286308,1.3921,-0.23488,-0.068358,-0.070661
1,-999.0,-999.0,-999.0,7.510893,-0.35749,-0.

In [ ]:
# # Use when we need to consider timetamps

# # get timestamp and sort by time
# test_df[["site", "file", "timestamp"]] = test_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
# test_df = test_df.drop(columns=["site_path_timestamp", "site", "file"])
# test_df["timestamp"] = test_df["timestamp"].astype(int)
# # display(test_df.head())

# # sort by time
# train_df = train_df.sort_values(by=["file_id", "wifi_ts"])
# test_df = test_df.sort_values(by=["file_id", "timestamp"])
# # display(train_df.head(20))
# # display(test_df.head(20))
# # print(len(test_df.columns))